#### License

This code was developed by Ameya Bhamare. I have taken snippets from Dr. David W. McDonald. 
This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.0 - August 13, 2023

#### Imports

In [21]:
#
#    IMPORTS
# 
from tqdm import tqdm
#    Import some standard python modules
import os, json, time
#
#    The module pyproj is a standard module that can be installed using pip or your other favorite
#    installation tool. This module provides tools to convert between different geodesic coordinate systems
#    and for calculating distances between points (coordinates) in a specific geodesic system.
#
from pyproj import Transformer, Geod
#
#    The 'wildfire' module is a user module. This module is available from the course website. The module
#    includes one object, a Reader, that can be used to read the GeoJSON files associated with the
#    wildefire dataset. The module also contains a sample datafile that is GeoJSON compliant and that
#    contains a small number of California wildfires extracted from the main wildfire dataset.
#    
from wildfire.wildfire.Reader import Reader as WFReader
#
#    There is a GeoJSON reader that you might try if you wanted to read the data. It has its own quirks.
#    There will be an example below that reads the sample file "Wildfire_short_sample.json"
#    
import geojson
#

In [22]:
with open('feature_list_135k.json', 'r') as file:
    feature_list = json.load(file)

In [23]:
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0], coord[1])
        new_coord = lat, lon
        converted_ring.append(new_coord)
    return converted_ring

In [24]:
def average_distance_from_place_to_fire_perimeter(place=None, ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1], place[0], point[1], point[0])
        distances_in_meters.append(d[2])

    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup) / len(distances_in_miles_no_dup)
    return average

### I updated the CITY_LOCATIONS dictionary to my city Merced, CA

In [25]:
CITY_LOCATIONS = {'merced' : {'city':'Merced', 'latlon' : [37.325954, -120.499992]}}

In [26]:
place = CITY_LOCATIONS["merced"]

### Filtering Process
Including only those fire locations which are within 1250 miles of Merced, CA. Also including fires which happened post 1963.

### Smoke Estimate Calculation:
In the given dataset, the parameters which give quantitative estimates of the fire are:

The Area Burnt by the Fire (direct proportion)
The Distance of the Fire from North Platte (inverse proportion)
The Intensity of the Fire (direct proportion)
The fire intensity can be computed based on the type of the fire (Wildfire, Prescribed Fire etc.) The metadata of the geographic data reveals that wildfire has highest intensity since they are proper fires. Prescribed fires might be smaller fires, hence I assigned a multiplying factor the same and calculated as follows:

### Fire-Estimate = (Area * Intensity) / Distance

In [27]:
fire_intensity = dict()
fire_intensity['Wildfire'] = 2
fire_intensity['Likely Wildfire'] = 1.75
fire_intensity['Unknown - Likely Wildfire'] = 1.5
fire_intensity['Prescribed Fire'] = 1.25
fire_intensity['Unknown - Likely Prescribed Fire'] = 1

In [ ]:
"""
Process a list of wildfire features to extract relevant information, calculate distance from a specified place,
and estimate smoke based on wildfire size and intensity. Filters wildfires with a year of 1963 or above and a
distance within 1250 miles from Merced, CA.

Parameters:
-----------
- feature_list (list): List of wildfire features, each represented as a dictionary.
- place (dict): Dictionary containing latitude and longitude information for a specified place.

Returns:
--------
list: A list of dictionaries representing relevant attributes of wildfires that meet the specified criteria,
including distance from the specified place and smoke estimate.

Usage:
------
1. Provide the 'feature_list' containing wildfire features and the 'place' dictionary with latitude and longitude.
2. Execute the code to process the wildfire features, calculate distance, and estimate smoke.
3. The resulting 'rel_fires' list will contain relevant attributes of wildfires meeting the specified criteria.
"""

In [28]:
rel_fires = []
for wf_feature in tqdm(feature_list):
    try:
      wf_year = wf_feature['attributes']['Fire_Year']
      wf_name = wf_feature['attributes']['Listed_Fire_Names'].split(',')[0]
      wf_size = wf_feature['attributes']['GIS_Acres']
      wf_type = wf_feature['attributes']['Assigned_Fire_Type']
      ring_data = wf_feature['geometry']['rings'][0]
    except:
      print("Exception occurred, ring param was missing")
    # Checking if wildfire_year is 1963 or above
    if wf_year >= 1963:
      distance = average_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)
      # Checking if distance of wildfire (in miles) is within 1250 miles from Merced, CA
      if distance < 1250:
        ring = convert_ring_to_epsg4326(ring_data)
        wf_feature['attributes']['distance'] = distance
        # Smoke estimate calculation
        smoke_estimate = (wf_size * fire_intensity[wf_type]) / distance
        wf_feature['attributes']['smoke_estimate'] = smoke_estimate
        rel_fires.append(wf_feature['attributes'])

 81%|████████  | 109607/135061 [1:58:35<22:54, 18.52it/s]  

Exception occurred, ring param was missing


 82%|████████▏ | 110225/135061 [1:59:17<24:12, 17.10it/s]

Exception occurred, ring param was missing


 82%|████████▏ | 110643/135061 [1:59:45<17:04, 23.84it/s]

...

 93%|█████████▎| 125746/135061 [2:16:08<12:16, 12.65it/s]

Exception occurred, ring param was missing


 94%|█████████▍| 127493/135061 [2:18:04<07:37, 16.55it/s]

Exception occurred, ring param was missing


100%|██████████| 135061/135061 [2:29:33<00:00, 15.05it/s] 


In [29]:
with open('filtered_fires_1963_to_2023.json', 'w') as json_file:
    json.dump(rel_fires, json_file)